In [1]:
from torch import tensor 
from torch.autograd import Variable
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.metrics import roc_auc_score
"""k fold 대신 stratified k fold"""
from sklearn.model_selection import StratifiedKFold
import itertools
#from pytorch_tabnet.tab_model import TabNetClassifier
from pathlib import Path
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
import sys
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import torch.utils.data as td
#import utilities
from tqdm.autonotebook import tqdm, trange
from matplotlib import pyplot as plt
import time
from torch.nn.parameter import Parameter
from torch.distributions import Gamma
from utilities import log_sum_exp
import math
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau
from utilities import R2_value, log_sum_exp, parameters_to_vector, vector_to_parameters, getOpt
from torch.optim.lr_scheduler import StepLR, ExponentialLR, ReduceLROnPlateau
from time import time
from sklearn.metrics import accuracy_score, roc_auc_score

#bayesean Tabnet


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('group1_5_risk_EC_abs_total (1) (1).csv')
df

,Unnamed: 0,delta_E1,delta_E10,delta_E100,delta_E101,delta_E102,delta_E103,delta_E104,delta_E105,delta_E106,...,gamma_E91,gamma_E92,gamma_E93,gamma_E94,gamma_E95,gamma_E96,gamma_E97,gamma_E98,gamma_E99,target
0,0,8.186840,10.606450,44.688722,12.757568,4.279060,6.252396,7.404375,8.745515,8.742263,...,61.988657,29.291054,29.744932,136.190847,212.924658,81.294987,48.675846,24.726053,142.232636,1.0
1,1,25.939918,19.104370,16.145237,13.580193,5.161970,11.025384,7.649788,9.213589,6.471280,...,23.900143,14.315390,15.506638,94.855550,59.283250,44.557439,16.220062,13.037046,65.670841,1.0
2,2,153.159664,15.066106,6.662611,5.487291,4.630674,4.649062,4.682885,4.240100,3.813276,...,44.393759,59.016298,33.794492,70.474366,78.776522,34.908377,61.082163,45.893154,93.348860,0.0
3,3,7.089047,10.054340,25.154236,23.959194,14.029636,10.943584,8.306384,4.749801,1.830242,...,20.186578,11.595557,11.295946,64.130118,22.663702,21.716074,12.985209,11.825560,29.928711,1.0
4,4,33.236915,12.426685,6.418845,4.376580,4.513896,4.290389,4.220892,4.660313,6.759117,...,0.439690,0.102942,0.091856,0.881043,0.286805,0.377908,0.149718,0.116125,0.319130,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77,54.506310,8.534871,6.765330,2.731705,3.034995,5.870074,6.415011,5.530279,3.704629,...,0.108132,0.068317,0.056253,0.195592,0.190453,0.134718,0.072268,0.050515,0.087241,1.0
78,78,35.139320,24.022022,13.486429,4.989185,3.973933,3.638945,3.372632,3.027413,7.088790,...,0.080603,0.081616,0.050210,0.103784,0.064158,0.065894,0.057960,0.051233,0.082560,1.0
79,79,9.798020,10.395767,30.070915,5.071613,3.062017,2.705522,2.427417,1.669092,1.556226,...,0.046448,0.035920,0.034157,0.075519,0.047879,0.043430,0.040736,0.032956,0.049081,1.0
80,80,21.128716,15.409268,4.162855,3.932925,3.936626,3.710372,3.884445,6.257363,9.081596,...,0.062187,0.060051,0.057154,0.063969,0.070002,0.067328,0.064743,0.064991,0.222946,1.0


In [5]:
df = df.iloc[: , 1:]
df

,delta_E1,delta_E10,delta_E100,delta_E101,delta_E102,delta_E103,delta_E104,delta_E105,delta_E106,delta_E107,...,gamma_E91,gamma_E92,gamma_E93,gamma_E94,gamma_E95,gamma_E96,gamma_E97,gamma_E98,gamma_E99,target
0,8.186840,10.606450,44.688722,12.757568,4.279060,6.252396,7.404375,8.745515,8.742263,16.643758,...,61.988657,29.291054,29.744932,136.190847,212.924658,81.294987,48.675846,24.726053,142.232636,1.0
1,25.939918,19.104370,16.145237,13.580193,5.161970,11.025384,7.649788,9.213589,6.471280,24.463102,...,23.900143,14.315390,15.506638,94.855550,59.283250,44.557439,16.220062,13.037046,65.670841,1.0
2,153.159664,15.066106,6.662611,5.487291,4.630674,4.649062,4.682885,4.240100,3.813276,21.183522,...,44.393759,59.016298,33.794492,70.474366,78.776522,34.908377,61.082163,45.893154,93.348860,0.0
3,7.089047,10.054340,25.154236,23.959194,14.029636,10.943584,8.306384,4.749801,1.830242,31.439636,...,20.186578,11.595557,11.295946,64.130118,22.663702,21.716074,12.985209,11.825560,29.928711,1.0
4,33.236915,12.426685,6.418845,4.376580,4.513896,4.290389,4.220892,4.660313,6.759117,8.330469,...,0.439690,0.102942,0.091856,0.881043,0.286805,0.377908,0.149718,0.116125,0.319130,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,54.506310,8.534871,6.765330,2.731705,3.034995,5.870074,6.415011,5.530279,3.704629,6.814907,...,0.108132,0.068317,0.056253,0.195592,0.190453,0.134718,0.072268,0.050515,0.087241,1.0
78,35.139320,24.022022,13.486429,4.989185,3.973933,3.638945,3.372632,3.027413,7.088790,6.686735,...,0.080603,0.081616,0.050210,0.103784,0.064158,0.065894,0.057960,0.051233,0.082560,1.0
79,9.798020,10.395767,30.070915,5.071613,3.062017,2.705522,2.427417,1.669092,1.556226,7.140624,...,0.046448,0.035920,0.034157,0.075519,0.047879,0.043430,0.040736,0.032956,0.049081,1.0
80,21.128716,15.409268,4.162855,3.932925,3.936626,3.710372,3.884445,6.257363,9.081596,13.574650,...,0.062187,0.060051,0.057154,0.063969,0.070002,0.067328,0.064743,0.064991,0.222946,1.0


In [6]:

train_data = df
target = 'target'

In [7]:
features = [ col for col in train_data.columns if col != target] 

In [8]:
train_data = df
#test_data = test

NUM_FOLDS  = 10
# the number of feature that you want to show 
Num_feat = 20

target ='target'
#unused_feat = ['abcd_site','kfold']

# mri feature가 시작하는 column의 index 구하기
# np.where의 결과값이 array에 들어가기 때문에 방금 계산해 넣어놓은 [0]번째 값을 가져온다.
start_delta_index = np.where(train_data.columns.values == "delta_E11")[0][0]
start_theta_index = np.where(train_data.columns.values == "theta_E11")[0][0]
start_beta1_index = np.where(train_data.columns.values == "beta1_E11")[0][0]
start_beta2_index = np.where(train_data.columns.values == "beta2_E11")[0][0]
start_beta3_index = np.where(train_data.columns.values == "beta3_E11")[0][0]
start_gamma_index = np.where(train_data.columns.values == "gamma_E11")[0][0]



delta = list(train_data.columns[start_delta_index:start_theta_index])
theta = list(train_data.columns[start_theta_index:start_beta1_index])
beta1 = list(train_data.columns[start_beta1_index:start_beta2_index])
beta2 = list(train_data.columns[start_beta2_index:start_beta3_index])
beta3= list(train_data.columns[start_beta3_index:start_gamma_index])
gamma = list(train_data.columns[start_gamma_index:])





print(len(train_data), len(train_data.columns))
print(len(delta))
print(len(theta))
print(len(beta1))
print(len(beta2))
print(len(beta3))
print(len(gamma))

82 904
129
258
129
129
129
118


In [9]:
class Sparsemax(nn.Module):
    def __init__(self, dim=None):
        super(Sparsemax, self).__init__()
        self.dim = -1 if dim is None else dim

    def forward(self, input):
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)
        
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, device=device,step=1, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]
        zs_sparse = is_gt * zs
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)
        self.output = torch.max(torch.zeros_like(input), input - taus)
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)
        return output
    def backward(self, grad_output):
        dim = 1
        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))
        return self.grad_input

def initialize_non_glu(module,inp_dim,out_dim):
    gain = np.sqrt((inp_dim+out_dim)/np.sqrt(4*inp_dim))
    torch.nn.init.xavier_normal_(module.weight, gain=gain)
    
class GBN(nn.Module):
    def __init__(self,inp,vbs=128,momentum=0.01):
        super().__init__()
        self.bn = nn.BatchNorm1d(inp,momentum=momentum)
        self.vbs = vbs
    def forward(self,x):
        chunk = torch.chunk(x,max(1,x.size(0)//self.vbs),0)
        res = [self.bn(y) for y in chunk ]
        return torch.cat(res,0)

class GLU(nn.Module):
    def __init__(self,inp_dim,out_dim,fc=None,vbs=128):
        super().__init__()
        if fc:
            self.fc = fc
        else:
            self.fc = nn.Linear(inp_dim,out_dim*2)
        self.bn = GBN(out_dim*2,vbs=vbs) 
        self.od = out_dim
    def forward(self,x):
        x = self.bn(self.fc(x))
        return x[:,:self.od]*torch.sigmoid(x[:,self.od:])
    

class FeatureTransformer(nn.Module):
    def __init__(self,inp_dim,out_dim,shared,n_ind,vbs=128):
        super().__init__()
        first = True
        self.shared = nn.ModuleList()
        if shared:
            self.shared.append(GLU(inp_dim,out_dim,shared[0],vbs=vbs))
            first= False    
            for fc in shared[1:]:
                self.shared.append(GLU(out_dim,out_dim,fc,vbs=vbs))
        else:
            self.shared = None
        self.independ = nn.ModuleList()
        if first:
            self.independ.append(GLU(inp,out_dim,vbs=vbs))
        for x in range(first, n_ind):
            self.independ.append(GLU(out_dim,out_dim,vbs=vbs))
        self.scale = torch.sqrt(torch.tensor([.5],device=device))
    def forward(self,x):
        if self.shared:
            x = self.shared[0](x)
            for glu in self.shared[1:]:
                x = torch.add(x, glu(x))
                x = x*self.scale
        for glu in self.independ:
            x = torch.add(x, glu(x))
            x = x*self.scale
        return x
class AttentionTransformer(nn.Module):
    def __init__(self,inp_dim,out_dim,relax,vbs=128):
        super().__init__()
        self.fc = nn.Linear(inp_dim,out_dim)
        self.bn = GBN(out_dim,vbs=vbs)
#         self.smax = Sparsemax()
        self.r = torch.tensor([relax],device=device)
    def forward(self,a,priors):
        a = self.bn(self.fc(a))
        mask = torch.sigmoid(a*priors)
        priors =priors*(self.r-mask)
        return mask

class DecisionStep(nn.Module):
    def __init__(self,inp_dim,n_d,n_a,shared,n_ind,relax,vbs=128):
        super().__init__()
        self.fea_tran = FeatureTransformer(inp_dim,n_d+n_a,shared,n_ind,vbs)
        self.atten_tran = AttentionTransformer(n_a,inp_dim,relax,vbs)
    def forward(self,x,a,priors):
        mask = self.atten_tran(a,priors)
        loss = ((-1)*mask*torch.log(mask+1e-10)).mean()
        x = self.fea_tran(x*mask)
        return x,loss


class TabNet(nn.Module):
    def __init__(self,inp_dim,final_out_dim,n_d=2,n_a=2,n_shared=2,n_ind=2,n_steps=3,relax=1.3,vbs=128):
        super().__init__()
        
        if n_shared>0:
            self.shared = nn.ModuleList()
            self.shared.append(nn.Linear(inp_dim,2*(n_d+n_a)))
            for x in range(n_shared-1):
                self.shared.append(nn.Linear(n_d+n_a,2*(n_d+n_a)))
        else:
            self.shared=None
            
        self.first_step = FeatureTransformer(inp_dim,n_d+n_a,self.shared,n_ind) 
        self.steps = nn.ModuleList()
        for x in range(n_steps-1):
            self.steps.append(DecisionStep(inp_dim,n_d,n_a,self.shared,n_ind,relax,vbs))
        self.fc = nn.Linear(n_d,final_out_dim)
        self.bn = nn.BatchNorm1d(inp_dim)
        self.n_d = n_d
        self.sigm = nn.Sigmoid()
        
        #self.features = nn.Sequential()
        #self.features.add_module("BN1", nn.BatchNorm1d(inp_dim))
        
        
            
#         self.features.add_module("FirstStep", FeatureTransformer(inp_dim,n_d+n_a,self.shared,n_ind))
#         #self.steps = nn.ModuleList()
#         for x in range(n_steps-1):
#             self.features.add_module('Step%d' % x, DecisionStep(inp_dim,n_d,n_a,self.shared,n_ind,relax,vbs))
#         self.features.add_module('FC', nn.Linear(n_d, final_out_dim))
#         self.features.add_module('Sigmoid', nn.Sigmoid())
        
    def forward(self,x):
        x = self.bn(x)
        x_a = self.first_step(x)[:,self.n_d:]
        loss = torch.zeros(1).to(x.device)
        out = torch.zeros(x.size(0),self.n_d).to(x.device)
        priors = torch.ones(x.shape).to(x.device)
        for step in self.steps:
            x_te,l = step(x,x_a,priors)
            out += F.relu(x_te[:,:self.n_d])
            x_a = x_te[:,self.n_d:]
            loss += l
        out = self.fc(out)
        #out = self.sigm(out)
        #x = self.features(x)
        
        return out, loss
    
    def _count_parameters(self):
        n_params = 0
        for name, param in self.named_parameters():
            print(name)
            print(param.size())
            print(param.numel())
            n_params += param.numel()
            #print('num of parameters so far: {}'.format(n_params))

    def reset_parameters(self, verbose=False):
        for module in self.modules():
            # pass self, otherwise infinite loop
            if isinstance(module, self.__class__):
                continue
            if 'reset_parameters' in dir(module):
                if callable(module.reset_parameters):
                    module.reset_parameters()
                    #if verbose:
                        #print("Reset parameters in {}".format(module))

In [10]:
class BayesTabnet(nn.Module):
    def __init__(self, n_samples, device, model):
        super(BayesTabnet, self).__init__()
        self.n_samples = n_samples
        self.device = device
        self.w_prior_shape = 1
        self.w_prior_rate = 0.05
        self.beta_prior_shape = 2
        self.beta_prior_rate = 1.e-6
        
        instances = []
        for i in range(self.n_samples):
            new_instance = copy.deepcopy(model)
            new_instance.reset_parameters()
            #print('Reset parameters in model instance {}'.format(i+1))
            instances.append(new_instance)
        self.nets = nn.ModuleList(instances)
        del instances
        
        log_beta = Gamma(self.beta_prior_shape, self.beta_prior_rate).sample((self.n_samples,)).log()
        for i in range(self.n_samples):
            self.nets[i].log_beta = Parameter(log_beta[i])

        #print('Total number of parameters: {}'.format(self._num_parameters()))
        
        
        
    def _num_parameters(self):
        count = 0
        for name, param in self.named_parameters():
            count += param.numel()
        return count

    def __getitem__(self, idx):
        return self.nets[idx]

    @property
    def log_beta(self):
        return torch.tensor([self.nets[i].log_beta.item() 
            for i in range(self.n_samples)], device=self.device)

    def forward(self, input):
        output = []
        for i in range(self.n_samples):
            fout, floss = self.nets[i].forward(input)
            output.append(fout)
        output = torch.stack(output)

        return output

    def _log_joint(self, index, output, target, ntrain): #depends on beta prior
        #log_likelihood = ntrain / output.size(0) * (
        #                    - 0.5 * self.nets[index].log_beta.exp()
        #                    * (target - output).pow(2).sum()
        #                    + 0.5 * target.numel() * self.nets[index].log_beta)
       
        log_likelihood = ntrain / output.size(0) * (
                            - 0.5 * self.nets[index].log_beta.exp()
                            * -(target * torch.log(torch.sigmoid(output)+0.00001) + (1-target) * torch.log(1-torch.sigmoid(output)+0.00001)).sum()
                            + 0.5 * target.numel() * self.nets[index].log_beta)

        log_prob_prior_w = torch.tensor(0.).to(self.device)
        for param in self.nets[index].bn.parameters():
            log_prob_prior_w += \
                torch.log1p(0.5 / self.w_prior_rate * param.pow(2)).sum()
        for param in self.nets[index].first_step.parameters():
            log_prob_prior_w += \
                torch.log1p(0.5 / self.w_prior_rate * param.pow(2)).sum()
        for param in self.nets[index].steps.parameters():
            log_prob_prior_w += \
                torch.log1p(0.5 / self.w_prior_rate * param.pow(2)).sum()
        for param in self.nets[index].fc.parameters():
            log_prob_prior_w += \
                torch.log1p(0.5 / self.w_prior_rate * param.pow(2)).sum()
        log_prob_prior_w *= -(self.w_prior_shape + 0.5)
        log_prob_prior_log_beta = (self.beta_prior_shape * self.nets[index].log_beta \
                    - self.nets[index].log_beta.exp() * self.beta_prior_rate)
        return log_likelihood + log_prob_prior_w + log_prob_prior_log_beta

    def _compute_bce(self, input, target, size_average=True, out=False):
        output = self.forward(input)
        #log_beta = self.log_beta.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
        #log_2pi_S = torch.tensor(0.5 * target[0].numel() * math.log(2 * math.pi)
        #               + math.log(self.n_samples), device=self.device)
        #exponent = - 0.5 * (log_beta.exp() * ((target - output) ** 2)).view(
        #    self.n_samples, target.size(0), -1).sum(-1) \
        #           + 0.5 * target[0].numel() * self.log_beta.unsqueeze(-1)

        #nlp = - log_sum_exp(exponent, dim=0).mean() + log_2pi_S
        bce = F.binary_cross_entropy_with_logits(output.mean(0), target)

        if not size_average:
            bce *= target.numel()
            #nlp *= target.size(0)
        if not out:
            return bce
        else:
            return bce, output

    def predict(self, x_test):
        y = self.forward(x_test)
        y_pred_mean = y.mean(0)
        EyyT = (y ** 2).mean(0)
        EyEyT = y_pred_mean ** 2
        beta_inv = (- self.log_beta).exp()
        y_pred_var = beta_inv.mean() + EyyT - EyEyT

        return y_pred_mean, y_pred_var

    def propagate(self, mc_loader):
        output_size = mc_loader.dataset[0][1].size()
        cond_Ey = torch.zeros(self.n_samples, *output_size, device=self.device)
        cond_Eyy = torch.zeros_like(cond_Ey)

        for _, (x_mc, _) in enumerate(mc_loader):
            x_mc = x_mc.to(self.device)          
            y = self.forward(x_mc)
            cond_Ey += y.mean(1)
            cond_Eyy += y.pow(2).mean(1)
        cond_Ey /= len(mc_loader)
        cond_Eyy /= len(mc_loader)
        beta_inv = (- self.log_beta).exp()
        print('Noise variances: {}'.format(beta_inv))
        
        y_cond_pred_var = cond_Eyy - cond_Ey ** 2 \
                     + beta_inv.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

        return cond_Ey.mean(0), cond_Ey.var(0), \
               y_cond_pred_var.mean(0), y_cond_pred_var.var(0)
        

In [11]:
class SVGD(object):
    def __init__(self, train_loader, n_samples, lr, lr_noise, ntrain, btrain, out, device, bayes_net):
        self.bayes_net = bayes_net
        self.train_loader = train_loader
        self.n_samples = n_samples
        self.lr = lr
        self.lr_noise = lr_noise
        self.ntrain = ntrain
        self.batch_train_size = btrain
        self.out_channels = out
        self.device = device
        self.optimizers, self.schedulers = self._optimizers_schedulers(self.lr, self.lr_noise)

    def _squared_dist(self, X):
        XXT = torch.mm(X, X.t())
        XTX = XXT.diag()
        return -2.0 * XXT + XTX + XTX.unsqueeze(1)

    def _Kxx_dxKxx(self, X):
        squared_dist = self._squared_dist(X)
        l_square = 0.5 * squared_dist.median() / math.log(self.n_samples)
        Kxx = torch.exp(-0.5 / l_square * squared_dist)
        dxKxx = (Kxx.sum(1).diag() - Kxx).matmul(X) / l_square
        return Kxx, dxKxx
    
    def _optimizers_schedulers(self, lr, lr_noise):
        optimizers = []
        schedulers = []
        for i in range(self.n_samples):
            parameters = [{'params': [self.bayes_net[i].log_beta], 'lr': lr_noise}, {'params': self.bayes_net[i].bn.parameters()}, 
                          {'params': self.bayes_net[i].steps.parameters()}, {'params': self.bayes_net[i].fc.parameters()}]
            #torch.optim.Adam(model.parameters(),lr=2e-2,weight_decay=0.01)
            #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.99)  
            optimizer_i = torch.optim.Adam(parameters,lr=lr,weight_decay=0.01)
            optimizers.append(optimizer_i)
            schedulers.append(ReduceLROnPlateau(optimizer_i, mode='min', factor=0.1, patience=10, verbose=True))
            #schedulers.append(StepLR(optimizer_i, step_size = 10, gamma = 0.99))
        return optimizers, schedulers

    def train(self, epoch, logger, l1_pen, train_loader):
        self.bayes_net.train()
        bce_train = 0
        acc_train = 0
        auc_train = 0
        for batch_idx, (input_x, true_y) in enumerate(self.train_loader):
            input_x, true_y = input_x.to(self.device), true_y.to(self.device)
            true_y = torch.unsqueeze(true_y, 1)

            self.bayes_net.zero_grad()
            pred_y = torch.zeros_like(true_y)
            grad_log_joint = []
            theta = []
            log_joint = 0.

            for idx in range(self.n_samples):
                pred_y_i, pred_y_loss = self.bayes_net[idx].forward(input_x)
                pred_y += pred_y_i.detach()
                log_joint_i = self.bayes_net._log_joint(idx, pred_y_i, true_y, self.ntrain)
                log_joint_i += l1_pen*pred_y_loss.cpu().detach().numpy()[0]
                log_joint_i.backward()
                log_joint += log_joint_i.item()

                vec_param, vec_grad_log_joint = parameters_to_vector(self.bayes_net[idx].parameters(), both=True)
                grad_log_joint.append(vec_grad_log_joint.unsqueeze(0))
                theta.append(vec_param.unsqueeze(0))

            theta = torch.cat(theta)
            Kxx, dxKxx = self._Kxx_dxKxx(theta)
            grad_log_joint = torch.cat(grad_log_joint)
            grad_logp = torch.mm(Kxx, grad_log_joint)
            grad_theta = - (grad_logp + dxKxx) / self.n_samples

            for idx in range(self.n_samples):
                vector_to_parameters(grad_theta[idx], self.bayes_net[idx].parameters(), grad=True)
                self.optimizers[idx].step()
                #self.schedulers[idx].step()
            bce_train += F.binary_cross_entropy_with_logits(pred_y / self.n_samples, true_y).item()
            acc_train += accuracy_score(true_y.cpu(), np.where((pred_y.cpu()/self.n_samples).data < 0.5, 0, 1))
            #auc_train += roc_auc_score(y_true = true_y.cpu(), y_score = (pred_y.cpu()/self.n_samples).data)
            #r2_score += R2_value(pred_y / self.n_samples, true_y)

        avg_bce_train = bce_train / len(self.train_loader)
        avg_acc_train = acc_train / len(self.train_loader)
        #avg_auc_train = auc_train / len(self.train_loader)
        #r2_train = r2_score / len(self.train_loader)
        logger['bce_train'].append(bce_train)
        #logger['r2_train'].append(r2_train)
        logger['log_beta'].append(self.bayes_net.log_beta.mean().item())
        #print("epoch {}, training bce: {:.6f}, training acc: {:.6f}, training auc: {:.6f}".format(epoch, avg_bce_train, avg_acc_train, avg_auc_train))
        print("epoch {}, training bce: {:.6f}, training acc: {:.6f}".format(epoch, avg_bce_train, avg_acc_train))


        for idx in range(self.n_samples):
            self.schedulers[idx].step(avg_bce_train)


In [12]:
def test(model, epoch, logger, test_loader, model_load, model_state):
    if model_load == True:
      model.load_state_dict(torch.load(model_state))
    model.eval()
    start = time()
    
    bce_test = 0.
    #count = 0
    acc_test = 0.
    auc_test = 0.
    
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        bce, outputs = model._compute_bce(inputs, targets.unsqueeze(1), size_average=True, out=True)
        outputs_mean = outputs.mean(0)
        
        EyyT = (outputs ** 2).mean(0)
        EyEyT = outputs_mean ** 2
        outputs_noise_var = (- model.log_beta).exp().mean()
        outputs_var =  EyyT - EyEyT + outputs_noise_var
        
        acc_test += accuracy_score(targets.cpu(),np.where(outputs_mean.cpu().data < 0.5, 0, 1))
        auc_test += roc_auc_score(y_true = targets.cpu(), y_score = outputs_mean.cpu().data)
        
        bce_test += bce.item()

    bce_test = bce_test/len(test_loader)
    acc_test = acc_test/len(test_loader)
    auc_test = auc_test/len(test_loader)

    #if epoch == args.epochs:
    #    saveData(tar_save, out_save, err_save, uncer_save, args.save_dir)
    #    plotTestSample(out_save, tar_save, err_save, uncer_save, args.save_dir, args.num_Plot)

    stop = time()
    
    print('Test epoch: {}, bce: {:.6f}, acc: {:.6f}, auc: {:.6f}, time: {:.3f}'.format(epoch, bce_test, acc_test, auc_test, stop-start))

    if epoch % 1 == 0:
        #logger['r2_test'].append(r2_score)
        logger['bce_test'].append(bce_test)
        #logger['mnlp_test'].append(mnlp_test)
    
    return(model, acc_test, auc_test)

In [13]:
def feature(NUM_FOLDS, Num_feat, clf_res, test_data_processed, features):
    importance_res = []
    for i in clf_res:
        importance_clf =i.feature_importances_
        importance_res.append(importance_clf)
    
    importance=[importance_res[0][i]/NUM_FOLDS+importance_res[1][i]/NUM_FOLDS+ importance_res[2][i]/NUM_FOLDS+ importance_res[3][i]/NUM_FOLDS+ importance_res[4][i]/NUM_FOLDS for i in range(len(importance_res[1]))]
    
    #feat_name_sort=test_data_processed[features].columns[labels_importance]
    feat_name_sort = test_data_processed[features].columns
    important_features = pd.DataFrame([importance],columns = feat_name_sort, index=['Importance']) 
    important_features = important_features.transpose().sort_values(by=['Importance'], ascending=False)
    important_features = important_features.head(Num_feat)

    return important_features

In [14]:
def preprocessing (train_data, NUM_FOLDS, target):
    #imputation
    train_data_processed = train_data.fillna(0).reset_index()

    train_data_processed=train_data_processed.rename(columns = {'index' : 'subjectkey'})


    train_data["kfold"] = -1

    # frac: 전체 row 중 몇 %를 반환할 지 결정 -> frac=1을 설정해서 모든 데이터를 반환
    # random_state: 추후 이것과 동일한 샘플링을 재현하기 위함
    # sample: 데이터에서 임의의 샘플 선정 -> frac=1이면 전체 data의 순서만 임의로 바뀜
    train_data_processed = train_data_processed.sample(frac=1,random_state=2020).reset_index(drop=True)
    
    kf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state =0)
    
    # enumerate: 각 split된 data set 순서대로 index를 함께 반환
        # ex. fold 0 일 때, 80%는 trn_, 20%는 val_
    for fold, (trn_, val_) in enumerate(kf.split(X=train_data_processed, y=train_data_processed[target])):

        # 'kfold' 칼럼은 cross validation할 때 fold 순서를 지정해놓은 것. 
        train_data_processed.loc[val_, 'kfold'] = fold
    
    print("done preprocessing")
    return train_data_processed

train_data_processed = preprocessing(train_data,10,target)

done preprocessing


In [15]:
def find_bestpar(fold, train_data_processed, features, target):
    
    # Store maximum auc
    max_auc= 0
    # Store maximum hypterparameter set
    max_hy = []
    """
    # define hyperparameter space : learning rate, 
    n_ = [4,8,16]                              # 
    lr_ = [2e-2, 1e-2, 5e-3, 2e-3, 1e-3, 1e-4] # learning rate
    w_ = [0.01, 0.001, 0.0001]                 # weight decay
    g_ = [0.95, 0.99, 0.9]                     # scheduler params - gamma
    ss_ = [10, 20, 30]                         # scheduler params - step_size
    
    # Orginal hyperparameter space 
    """
    # define hyperparameter space (quick version)
    n_ = [4,16]
    lr_ = [2e-2,1e-3]
    w_ = [0.01,0.001]
    
    
    all_ = [n_, lr_, w_]
    h_space = [s for s in itertools.product(*all_)]
    
    for hy in tqdm(h_space):
        """===================Cross Validation==================="""
        
        """validation & test 결과"""
        valid_res = []

        for i in range(fold):
            #print("fold ", i)
            # 5개의 fold 사용했으므로 변수 fold 값은 차례대로 0,1,2,3,4 중 하나

            df_train = train_data_processed[train_data_processed.kfold != i]  # 5개 중 4개 train에 할당
            df_valid = train_data_processed[train_data_processed.kfold == i]  # 5개 중 1개 validation에 할당

            X_train = df_train[features].values
            Y_train = df_train[target].values
            
            X_valid = df_valid[features].values
            Y_valid = df_valid[target].values

            train_x = torch.Tensor(X_train)
            train_y = torch.Tensor(Y_train.flatten())
            train_ds = utils.TensorDataset(train_x,train_y)
            #class_sample_count = np.array(
            #                           [len(np.where(np.transpose(y_train.flatten()) == t)[0]) for t in np.unique(np.transpose(y_train.flatten()))])
            #class_weight = 1/class_sample_count
            #samples_weight = np.array([class_weight[t] for t in np.transpose(y_train.flatten())])
            #samples_weight = torch.from_numpy(samples_weight)
            #sampler = td.WeightedRandomSampler(samples_weight, len(samples_weight))
            train_loader = td.DataLoader(train_ds, batch_size=64,
                shuffle=False, num_workers=1)

            val_x = torch.Tensor(X_valid)
            val_y = torch.Tensor(Y_valid.flatten())
            val_ds = utils.TensorDataset(val_x,val_y)
            val_loader = td.DataLoader(val_ds, batch_size=len(Y_valid.flatten()),
                shuffle=False, num_workers=1)

                        
            model_TabNet = TabNet(inp_dim = X_train.shape[1], final_out_dim = 1, n_d=hy[0],n_a=hy[0],n_shared=2,n_ind=2,n_steps=3,relax=1.3,vbs=128).to(device)
            model = BayesTabnet(n_samples = 10, device = device, model = model_TabNet).to(device)
            criterion, optimizer, scheduler, logger = getOpt(lrs = 'ReduceLROnPlateau', lr = hy[1], weight_decay = hy[2], model = model)
            model_SVGD = SVGD(train_loader = train_loader, n_samples = 10, lr = hy[1], lr_noise = hy[2], ntrain = X_train.shape[0], btrain = 64, out = 1, device = device, bayes_net = model)
            best_val_auc = 0.
            best_val_epoch = 1
            for epoch in range(1,  11):
              model_SVGD.train(epoch, logger, 0.001, train_loader)
              with torch.no_grad():
                val_model, val_acc, val_auc = test(model, epoch, logger, val_loader, False, None)
                if val_auc > best_val_auc:
                  torch.save(val_model.state_dict(), 'BayesTabNetMRI_Fold'+str(epoch)+'_.pt')
                  best_val_auc = val_auc
                  best_val_epoch = epoch
                
            valid_res.append(best_val_auc)
            print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% best_val_auc)
    
        """valid의 평균, 표준편차 출력"""
        print("=====parameter별 valid, test score=====")
        print("Validation 평균: %3f"%np.mean(valid_res))

        if np.mean(valid_res)>max_auc:
            print("Find new maximum AUC!!")
            max_hy = hy
            max_auc = np.mean(valid_res)
    
    return max_hy

In [16]:
def bestpar_tuning(fold, train_data_processed, max_hy, features, target):
    hy = max_hy
    print("Max hy:" ,hy)
    

    """validation"""
    valid_res = []

    clf_res = []
    preds_prob_res = []
    
    """해당 버전에 추가된 코드"""    
    y_valid_true = []
    y_valid_pred = []
    
    y_valid_subject = []

    
    for i in range(fold):

        df_train = train_data_processed[train_data_processed.kfold != i]  # 5개 중 4개 train에 할당
        df_valid = train_data_processed[train_data_processed.kfold == i]  # 5개 중 1개 validation에 할당

        X_train = df_train[features].values
        Y_train = df_train[target].values
        
        X_valid = df_valid[features].values
        Y_valid = df_valid[target].values

        y_valid_subject.append(df_valid['subjectkey'].values)

        
        """해당 버전 추가 코드"""
        y_valid_true.append(Y_valid)  

        train_x = torch.Tensor(X_train)
        train_y = torch.Tensor(Y_train.flatten())
        train_ds = utils.TensorDataset(train_x,train_y)
        
        #class_sample_count = np.array(
        #                           [len(np.where(np.transpose(y_train.flatten()) == t)[0]) for t in np.unique(np.transpose(y_train.flatten()))])
        #class_weight = 1/class_sample_count
        #samples_weight = np.array([class_weight[t] for t in np.transpose(y_train.flatten())])
        #samples_weight = torch.from_numpy(samples_weight)
        #sampler = td.WeightedRandomSampler(samples_weight, len(samples_weight))
        train_loader = td.DataLoader(train_ds, batch_size=64,
            shuffle=False, num_workers=1)

        val_x = torch.Tensor(X_valid)
        val_y = torch.Tensor(Y_valid.flatten())
        val_ds = utils.TensorDataset(val_x,val_y)
        val_loader = td.DataLoader(val_ds, batch_size=len(Y_valid.flatten()),
            shuffle=False, num_workers=1)
        

                    

        model_TabNet = TabNet(inp_dim = X_train.shape[1], final_out_dim = 1, n_d=hy[0],n_a=hy[0],n_shared=2,n_ind=2,n_steps=3,relax=1.3,vbs=128).to(device)
        model = BayesTabnet(n_samples = 10, device = device, model = model_TabNet).to(device)
        criterion, optimizer, scheduler, logger = getOpt(lrs = 'ReduceLROnPlateau', lr = hy[1], weight_decay = hy[2], model = model)
        model_SVGD = SVGD(train_loader = train_loader, n_samples = 10, lr = hy[1], lr_noise = hy[2], ntrain = X_train.shape[0], btrain = 64, out = 1, device = device, bayes_net = model)
        best_val_auc = 0.
        best_val_epoch = 1
        for epoch in range(1,  11):
            model_SVGD.train(epoch, logger, 0.001, train_loader)
            with torch.no_grad():
              val_model, val_acc, val_auc = test(model, epoch, logger, val_loader, False, None)
              if val_auc > best_val_auc:
                torch.save(val_model.state_dict(), 'BayesTabNetMRI_Fold'+str(epoch)+'_.pt')
                best_val_auc = val_auc
                best_val_epoch = epoch
        
        
        print('Loading Weights from Epoch: %d'%best_val_epoch)
        model_weight_path = 'BayesTabNetMRI_Fold'+str(best_val_epoch)+'_.pt'
        model.load_state_dict(torch.load(model_weight_path))
        
        valid_res.append(best_val_auc)
        print('[%3d/%4d] '%(i+1, fold),'Valid score: %2f'% best_val_auc)
        clf_res.append(model) #해당 fold에서 최고로 성능이 좋은 모델 append 

    """valid의 평균 출력"""
    print("Validation 평균: %3f"%np.mean(valid_res))
    
    #return test_auc,test_acc ,valid_result, clf_res, preds_prob, X_test, Y_test, y_valid_pred, y_test_pred, y_valid_subject, y_test_subject

    return valid_res


In [17]:
def run(train_data_processed, fold, Num_feat, features, target):
    name_test = train_data_processed['subjectkey'].values
    print("-------------------------------Training Begining-------------------------------")
    n_ = [4,8,16]
    lr_ = [2e-2, 1e-2, 1e-3, 1e-4]
    w_ = [0.01, 0.001, 0.0001]
    all_ = [n_, lr_, w_]
    h_space = [s for s in itertools.product(*all_)]
    
    # Start training
    max_hy = find_bestpar(fold, train_data_processed, features, target)
    # if you want to just test the code, you should use this
    #max_hy = h_space[0]
    #print("Found maximum hyperparmeter, now work with that")

    print("-------------------------------Testing Begining-------------------------------")
    valid_auc = bestpar_tuning(fold, train_data_processed, max_hy, features, target)
    
    #print("-------------------------------Important Feature-------------------------------")
    #import_feat=feature(Num_feat, clf_res, test_data_processed, features)
    #import_feat=0
    #preds_val_prob = clf.predict_proba(X_valid)

    return valid_auc

In [18]:
class model():
    def __init__(self, train_data_processed, Num_FOLDS, Num_feat, features,target):
        valid_auc = run(train_data_processed,Num_FOLDS, Num_feat,features, target)
    
    
        self.train_data_processed = train_data_processed
        self.valid_auc = valid_auc
    

In [19]:

features_all= [col for col in train_data_processed.columns if col in delta + theta+ beta1 +beta2 + beta3 + gamma]

In [20]:
addiction = model(train_data_processed, NUM_FOLDS, Num_feat, features_all, target)

-------------------------------Training Begining-------------------------------


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 1, training bce: 0.677347, training acc: 0.638021
Test epoch: 1, bce: 0.724514, acc: 0.555556, auc: 0.350000, time: 1.094
epoch 2, training bce: 0.612295, training acc: 0.638021
Test epoch: 2, bce: 0.730756, acc: 0.555556, auc: 0.450000, time: 1.094
epoch 3, training bce: 0.567509, training acc: 0.645833
Test epoch: 3, bce: 0.756986, acc: 0.555556, auc: 0.400000, time: 1.091
epoch 4, training bce: 0.532405, training acc: 0.661458
Test epoch: 4, bce: 0.778002, acc: 0.555556, auc: 0.450000, time: 1.141
epoch 5, training bce: 0.497442, training acc: 0.717014
Test epoch: 5, bce: 0.799884, acc: 0.555556, auc: 0.450000, time: 1.100
epoch 6, training bce: 0.463057, training acc: 0.780382
Test epoch: 6, bce: 0.806699, acc: 0.555556, auc: 0.500000, time: 1.118
epoch 7, training bce: 0.427117, training acc: 0.788194
Test epoch: 7, bce: 0.811020, acc: 0.555556, auc: 0.550000, time: 1.136
epoch 8, training bce: 0.390806, training acc: 0.796007
Test epoch: 8, bce: 0.822956, acc: 0.555556, auc

Test epoch: 3, bce: 0.849593, acc: 0.500000, auc: 0.400000, time: 1.086
epoch 4, training bce: 0.504546, training acc: 0.762500
Test epoch: 4, bce: 0.875438, acc: 0.500000, auc: 0.466667, time: 1.085
epoch 5, training bce: 0.475578, training acc: 0.812500
Test epoch: 5, bce: 0.881816, acc: 0.500000, auc: 0.600000, time: 1.084
epoch 6, training bce: 0.449793, training acc: 0.812500
Test epoch: 6, bce: 0.886258, acc: 0.500000, auc: 0.600000, time: 1.085
epoch 7, training bce: 0.425245, training acc: 0.820312
Test epoch: 7, bce: 0.892823, acc: 0.500000, auc: 0.600000, time: 1.115
epoch 8, training bce: 0.401829, training acc: 0.820312
Test epoch: 8, bce: 0.907779, acc: 0.500000, auc: 0.733333, time: 1.100
epoch 9, training bce: 0.379797, training acc: 0.828125
Test epoch: 9, bce: 0.930112, acc: 0.500000, auc: 0.733333, time: 1.075
epoch 10, training bce: 0.358829, training acc: 0.828125
Test epoch: 10, bce: 0.958248, acc: 0.500000, auc: 0.733333, time: 1.079
[  7/  10]  Valid score: 0.733

Test epoch: 5, bce: 0.635283, acc: 0.625000, auc: 0.600000, time: 1.081
epoch 6, training bce: 0.448413, training acc: 0.812500
Test epoch: 6, bce: 0.637354, acc: 0.625000, auc: 0.666667, time: 1.100
epoch 7, training bce: 0.418092, training acc: 0.812500
Test epoch: 7, bce: 0.651989, acc: 0.625000, auc: 0.666667, time: 1.105
epoch 8, training bce: 0.389732, training acc: 0.812500
Test epoch: 8, bce: 0.690021, acc: 0.625000, auc: 0.666667, time: 1.108
epoch 9, training bce: 0.361793, training acc: 0.828125
Test epoch: 9, bce: 0.728697, acc: 0.625000, auc: 0.666667, time: 1.090
epoch 10, training bce: 0.333493, training acc: 0.828125
Test epoch: 10, bce: 0.766564, acc: 0.625000, auc: 0.666667, time: 1.087
[  3/  10]  Valid score: 0.666667
epoch 1, training bce: 0.652691, training acc: 0.646875
Test epoch: 1, bce: 0.792704, acc: 0.500000, auc: 0.066667, time: 1.080
epoch 2, training bce: 0.584906, training acc: 0.654687
Test epoch: 2, bce: 0.814013, acc: 0.500000, auc: 0.066667, time: 1.

...

epoch 3, training bce: 0.528354, training acc: 0.820312
Test epoch: 3, bce: 0.699654, acc: 0.625000, auc: 0.400000, time: 1.035
epoch 4, training bce: 0.499365, training acc: 0.835938
Test epoch: 4, bce: 0.699387, acc: 0.625000, auc: 0.400000, time: 1.027
epoch 5, training bce: 0.476177, training acc: 0.835938
Test epoch: 5, bce: 0.698350, acc: 0.625000, auc: 0.400000, time: 1.025
epoch 6, training bce: 0.455278, training acc: 0.835938
Test epoch: 6, bce: 0.696843, acc: 0.625000, auc: 0.400000, time: 1.023
epoch 7, training bce: 0.437257, training acc: 0.835938
Test epoch: 7, bce: 0.695235, acc: 0.625000, auc: 0.400000, time: 1.051
epoch 8, training bce: 0.421049, training acc: 0.835938
Test epoch: 8, bce: 0.693439, acc: 0.625000, auc: 0.466667, time: 1.048
epoch 9, training bce: 0.405892, training acc: 0.843750
Test epoch: 9, bce: 0.691390, acc: 0.625000, auc: 0.466667, time: 1.060
epoch 10, training bce: 0.390896, training acc: 0.843750
Test epoch: 10, bce: 0.689431, acc: 0.625000, a

Test epoch: 5, bce: 0.706072, acc: 0.625000, auc: 0.600000, time: 1.025
epoch 6, training bce: 0.479891, training acc: 0.843750
Test epoch: 6, bce: 0.704196, acc: 0.625000, auc: 0.600000, time: 1.048
epoch 7, training bce: 0.458970, training acc: 0.851562
Test epoch: 7, bce: 0.702117, acc: 0.625000, auc: 0.600000, time: 1.030
epoch 8, training bce: 0.439858, training acc: 0.859375
Test epoch: 8, bce: 0.699883, acc: 0.625000, auc: 0.600000, time: 1.032
epoch 9, training bce: 0.421297, training acc: 0.859375
Test epoch: 9, bce: 0.697636, acc: 0.625000, auc: 0.600000, time: 1.024
epoch 10, training bce: 0.403244, training acc: 0.859375
Test epoch: 10, bce: 0.695708, acc: 0.625000, auc: 0.600000, time: 1.034
[  9/  10]  Valid score: 0.600000
epoch 1, training bce: 0.686372, training acc: 0.604688
Test epoch: 1, bce: 0.673764, acc: 0.625000, auc: 1.000000, time: 1.058
epoch 2, training bce: 0.582931, training acc: 0.693750
Test epoch: 2, bce: 0.674986, acc: 0.625000, auc: 0.933333, time: 1.

Test epoch: 7, bce: 0.686272, acc: 0.625000, auc: 0.533333, time: 1.028
epoch 8, training bce: 0.403352, training acc: 0.843750
Test epoch: 8, bce: 0.685378, acc: 0.625000, auc: 0.466667, time: 1.032
epoch 9, training bce: 0.387214, training acc: 0.843750
Test epoch: 9, bce: 0.684474, acc: 0.625000, auc: 0.466667, time: 1.030
epoch 10, training bce: 0.371412, training acc: 0.843750
Test epoch: 10, bce: 0.683520, acc: 0.625000, auc: 0.466667, time: 1.016
[  5/  10]  Valid score: 0.600000
epoch 1, training bce: 0.684014, training acc: 0.689063
Test epoch: 1, bce: 0.667835, acc: 0.625000, auc: 0.733333, time: 1.056
epoch 2, training bce: 0.587098, training acc: 0.754687
Test epoch: 2, bce: 0.662310, acc: 0.625000, auc: 0.666667, time: 1.031
epoch 3, training bce: 0.540872, training acc: 0.820312
Test epoch: 3, bce: 0.656856, acc: 0.625000, auc: 0.666667, time: 1.020
epoch 4, training bce: 0.510859, training acc: 0.820312
Test epoch: 4, bce: 0.652098, acc: 0.625000, auc: 0.800000, time: 1.

epoch 9, training bce: 0.328558, training acc: 0.796007
Test epoch: 9, bce: 0.949665, acc: 0.666667, auc: 0.450000, time: 1.034
epoch 10, training bce: 0.288979, training acc: 0.851562
Test epoch: 10, bce: 1.022000, acc: 0.666667, auc: 0.450000, time: 1.033
Loading Weights from Epoch: 2
[  1/  10]  Valid score: 0.600000
epoch 1, training bce: 0.665744, training acc: 0.685764
Test epoch: 1, bce: 0.692763, acc: 0.555556, auc: 0.550000, time: 1.026
epoch 2, training bce: 0.591536, training acc: 0.693576
Test epoch: 2, bce: 0.702136, acc: 0.555556, auc: 0.450000, time: 1.023
epoch 3, training bce: 0.544391, training acc: 0.756944
Test epoch: 3, bce: 0.705078, acc: 0.555556, auc: 0.400000, time: 1.055
epoch 4, training bce: 0.510336, training acc: 0.756944
Test epoch: 4, bce: 0.707114, acc: 0.555556, auc: 0.300000, time: 1.045
epoch 5, training bce: 0.481070, training acc: 0.756944
Test epoch: 5, bce: 0.719890, acc: 0.555556, auc: 0.300000, time: 1.027
epoch 6, training bce: 0.452610, train

epoch 10, training bce: 0.397175, training acc: 0.812500
Test epoch: 10, bce: 0.632880, acc: 0.500000, auc: 0.733333, time: 1.027
Loading Weights from Epoch: 6
[  7/  10]  Valid score: 0.733333
epoch 1, training bce: 0.692782, training acc: 0.646875
Test epoch: 1, bce: 0.620750, acc: 0.625000, auc: 0.800000, time: 1.026
epoch 2, training bce: 0.631102, training acc: 0.646875
Test epoch: 2, bce: 0.620479, acc: 0.625000, auc: 0.866667, time: 1.031
epoch 3, training bce: 0.570728, training acc: 0.662500
Test epoch: 3, bce: 0.623240, acc: 0.625000, auc: 0.933333, time: 1.027
epoch 4, training bce: 0.531605, training acc: 0.662500
Test epoch: 4, bce: 0.614555, acc: 0.625000, auc: 0.800000, time: 1.045
epoch 5, training bce: 0.498581, training acc: 0.662500
Test epoch: 5, bce: 0.596366, acc: 0.625000, auc: 0.733333, time: 1.029
epoch 6, training bce: 0.469090, training acc: 0.720313
Test epoch: 6, bce: 0.576345, acc: 0.625000, auc: 0.866667, time: 1.025
epoch 7, training bce: 0.439926, train